In [20]:
import numpy as np
import pandas as pd
from random import seed
from random import randrange

In [21]:
def load_data():
    numpy_cov_matrix = np.genfromtxt("hwk2_datasets/DS1_Cov.txt", delimiter=',')[:,:-1]
    cov_matrix = pd.DataFrame(numpy_cov_matrix)
    negative_mean = np.genfromtxt("hwk2_datasets/DS1_m_0.txt", delimiter=',')[:-1]
    positive_mean = np.genfromtxt("hwk2_datasets/DS1_m_1.txt", delimiter=',')[:-1]
    return cov_matrix, negative_mean, positive_mean

cov_matrix, negative_mean, positive_mean = load_data()



In [22]:
def generate_data(mean, cov, size):
    data = np.random.multivariate_normal(mean, cov, size)
    return data

In [23]:
negative_data = generate_data(negative_mean, cov_matrix, 2000)
positive_data = generate_data(positive_mean, cov_matrix, 2000)

negative_examples = pd.DataFrame(negative_data) 
negative_examples[20] = 'negative'

positive_examples = pd.DataFrame(positive_data)
positive_examples[20] = 'positive'

In [24]:
def train_valid_test_split(dataset, train_split, valid_split, test_split):
    if (train_split + valid_split + test_split) != 1:
        raise ValueError("invalid size for train_split, valid_split, test_split")
    num_rows = dataset.shape[0]
    num_cols = dataset.shape[1]
    train = list()
    valid = list()
    dataset_copy = list(dataset)
    
    train_size = train_split*num_rows
    valid_size = valid_split*num_rows
    
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
        
    while len(valid) < valid_size:
        index = randrange(len(dataset_copy))
        valid.append(dataset_copy.pop(index))
        
    return np.array(train), np.array(valid), np.array(dataset_copy)
     


In [25]:
def create_datasets(neg_ex, pos_ex):
    # train/valid/test split
    seed(1)
    # negative splits
    neg_train, neg_valid, neg_test = train_valid_test_split(neg_ex.values, 0.6, 0.2, 0.2)

    # positive splits
    pos_train, pos_valid, pos_test = train_valid_test_split(pos_ex.values, 0.6, 0.2, 0.2)

    # create datasets
    train = np.concatenate((neg_train, pos_train),axis=0)
    np.random.shuffle(train) 
    train = pd.DataFrame(train)
    
    valid = np.concatenate((neg_valid, pos_valid),axis=0)
    np.random.shuffle(valid) 
    valid = pd.DataFrame(valid)
    
    test = np.concatenate((neg_test, pos_test),axis=0)
    np.random.shuffle(test) 
    test = pd.DataFrame(test)
    
    return train, valid, test

DS1_train, DS1_valid, DS1_test = create_datasets(negative_examples, positive_examples)

DS1_train.to_csv("DS1_train.csv")
DS1_valid.to_csv("DS1_valid.csv")
DS1_test.to_csv("DS1_test.csv")

In [26]:
def split_data_x_y(data):
    num_rows = data.shape[0]
    num_cols = data.shape[1]
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    return X, y


In [27]:
X_train, y_train_alpha = split_data_x_y(DS1_train)
y_train = y_train_alpha.replace(['negative', 'positive'], [0, 1])

X_valid, y_valid_alpha = split_data_x_y(DS1_valid)
y_valid = y_valid_alpha.replace(['negative', 'positive'], [0, 1])

X_test, y_test_alpha = split_data_x_y(DS1_test)
y_test = y_test_alpha.replace(['negative', 'positive'], [0, 1])

In [28]:
def calculate_pi_prior(y):
    y_copy = list(y)
    num_neg = y_copy.count(0)
    return num_neg/len(y_copy)

# negative
def calculate_mean_0(X, y):
    num_rows_x = X.shape[0]
    num_cols_x = X.shape[1]
    y_copy = list(y)
    num_ex = len(y_copy)
    
    num_neg = y_copy.count(0)
    x_given_0 = np.zeros(num_cols_x)
    
    for i in range(num_ex):
        xi = X[i,:]
        yi = y[i]
        if yi == 0:
            x_given_0 = x_given_0 + xi
    return (x_given_0/num_neg).astype(np.float32)

# positive
def calculate_mean_1(X, y):
    num_rows_x = X.shape[0]
    num_cols_x = X.shape[1]
    y_copy = list(y)
    num_ex = len(y_copy)
    
    num_pos = y_copy.count(1)
    x_given_1 = np.zeros(num_cols_x)
    
    for i in range(num_ex):
        xi = X[i,:]
        yi = y[i]
        if yi == 1:
            x_given_1 = x_given_1 + xi
    return (x_given_1/num_pos).astype(np.float32)

def calculate_sigma(X, y, m0, m1):
    num_rows_x = X.shape[0]
    num_cols_x = X.shape[1]
    y_copy = list(y)
    num_ex = len(y_copy)
    num_0 = y_copy.count(0)
    num_1 = y_copy.count(1)
    N_0 = num_0/num_ex
    N_1 = num_1/num_ex
    s_0 = np.zeros((num_cols_x, num_cols_x))
    s_1 = np.zeros((num_cols_x, num_cols_x))
    for i in range(num_ex):
        xi = np.array(X[i,:], dtype = np.float32)
        yi = y[i]
        if yi == 1:
            xm = np.reshape((xi-m0), (-1, 1))
            sqr_pos = np.dot(xm,xm.T)
            s_1 = s_1 + sqr_pos
        else:
            xm = np.reshape((xi-m1), (-1, 1))
            sqr_neg = np.dot(xm,xm.T)
            s_0 = s_0 + sqr_neg
    return N_0*((1/num_0)*s_0) + N_1*((1/num_1)*s_1)

def gda_parameters_maximum_likelihood(X, y):
    pi_prior = calculate_pi_prior(y)
    m0 = calculate_mean_0(X, y)
    m1 = calculate_mean_1(X, y)
    sigma = calculate_sigma(X, y, m0, m1)
    return pi_prior, m0, m1, sigma


In [29]:
pi_prior, m0, m1, sigma = gda_parameters_maximum_likelihood(X_train.values, y_train.values)

In [30]:
def prob_y(y, pi_prior):
    if y == 0:
        return pi_prior
    else:
        return 1-pi_prior

def pdf(x, mu, sigma):
    xm=np.reshape((x-mu), (-1, 1))
    dim = x.shape[0]
    pi = np.pi
    constant = 1./(((2*pi)**(dim/2))*np.linalg.det(sigma)**-0.5)
    return constant*np.exp(-(np.dot(np.dot(xm.T, np.linalg.inv(sigma)), xm))/2.)


In [31]:


def find_weights(m0, m1, sigma, cl, pi_pr):
    sigma_inv = np.linalg.inv(sigma)
    if(cl == 0):
        mum = np.reshape((m0-m1), (-1, 1))
        m0_r = np.reshape(m0, (-1, 1))
        m1_r = np.reshape(m1, (-1, 1))
        w = np.dot(sigma_inv, mum)
        w0 = -0.5*np.dot(np.dot(m0_r.T, sigma_inv), m0_r) + 0.5*np.dot(np.dot(m1_r.T, sigma_inv), m1_r) + np.log(prob_y(0, pi_pr)/prob_y(1, pi_pr))
        return w, w0
    else:
        mum = np.reshape((m1-m0), (-1, 1))
        m0_r = np.reshape(m0, (-1, 1))
        m1_r = np.reshape(m1, (-1, 1))
        w = np.dot(sigma_inv, mum)
        w0 = -0.5*np.dot(np.dot(m1_r.T, sigma_inv), m1_r) + 0.5*np.dot(np.dot(m0_r.T, sigma_inv), m0_r) + np.log(prob_y(1, pi_pr)/prob_y(0, pi_pr))
        return w, w0
        

# p(Ck|x)
def predict(xi, m0, m1, sigma, w_0, w0_0, w_1, w0_1):
    aa0 = np.dot(w_0.T, xi) + w0_0
    aa1 = np.dot(w_1.T, xi) + w0_1
    sig0 = 1 / (1 + np.exp(-aa0))
    sig1 = 1 / (1 + np.exp(-aa1))
    if(sig0 > sig1):
        return 0
    else:
        return 1

def classify(X, m0, m1, sigma, pi_pr):
    num_rows_x = X.shape[0]
    predictions = np.zeros(num_rows_x)
    w_0, w0_0 = find_weights(m0,m1,sigma, 0, pi_pr)
    w_1, w0_1 = find_weights(m0,m1,sigma, 1, pi_pr)
    
    print("Class 1:")
    print(w0_1)
    print()
    print(w_1)
    print()
    
    print("Class 0:")
    print(w0_0)
    print()
    print(w_0)
    print()
    
    for i in range(num_rows_x):
        xi = np.array(X[i,:], dtype=np.float32)
        yhat = predict(xi, m0, m1, sigma, w_0, w0_0, w_1, w0_1)
        predictions[i] = yhat
    return predictions



In [32]:
predictions = classify(X_test.values, m0, m1, sigma, pi_prior)

Class 1:
[[-2.16762388]]

[[-1.1498975 ]
 [ 0.67030086]
 [ 0.44289768]
 [ 0.2360709 ]
 [ 0.77487325]
 [ 0.34207348]
 [-1.3237704 ]
 [ 1.91948175]
 [ 2.29204001]
 [-0.7314653 ]
 [ 1.04502041]
 [ 0.95466482]
 [-1.22530663]
 [-1.03988365]
 [ 0.44911846]
 [-1.04061717]
 [-2.31835761]
 [ 0.55848259]
 [ 0.04508336]
 [ 0.40793269]]

Class 0:
[[2.16762388]]

[[ 1.1498975 ]
 [-0.67030086]
 [-0.44289768]
 [-0.2360709 ]
 [-0.77487325]
 [-0.34207348]
 [ 1.3237704 ]
 [-1.91948175]
 [-2.29204001]
 [ 0.7314653 ]
 [-1.04502041]
 [-0.95466482]
 [ 1.22530663]
 [ 1.03988365]
 [-0.44911846]
 [ 1.04061717]
 [ 2.31835761]
 [-0.55848259]
 [-0.04508336]
 [-0.40793269]]



In [33]:
def get_accuracy(predictions, actual):
    num_rows =  predictions.shape[0]
    num_correct = 0
    for i in range(num_rows):
        if(predictions[i] == actual[i]):
            num_correct = num_correct + 1
    return num_correct/num_rows

def get_confusion_matrix(pred, actual):
    y_actu = pd.Series(actual, name='Actual')
    y_pred = pd.Series(pred, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred)
    return df_confusion

def get_precision(conf_matrix):
    return conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])

def get_recall(conf_matrix):
    return conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[1,0])

def get_f_measure(p, r):
    return (2*p*r)/(p+r)



In [34]:
confusion_matrix = get_confusion_matrix(predictions, y_test.values)

accuracy = get_accuracy(predictions, y_test.values)
precision = get_precision(confusion_matrix.values)
recall = get_recall(confusion_matrix.values)
f_measure = get_f_measure(precision, recall)

accuracy, precision, recall, f_measure

(0.96, 0.9675, 0.9532019704433498, 0.9602977667493796)

In [35]:
import operator 

def euclideanDistance(test_data, data, length):
    dist = 0
    for i in range(length):
        dist = dist + np.square(test_data[i] - data[i])
    return np.sqrt(dist)


In [36]:
def knn(test_instance, train, k):
    num_rows_train = train.shape[0]
    num_cols_test = train.shape[1]-1
    distances = {}
    for i in range(num_rows_train):
        dist = euclideanDistance(test_instance, train[i,:-1], num_cols_test)
        distances[i] = dist
    
    sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(sorted_distances[i][0])
    
    votes = {}
    for i in range(len(neighbors)):
        response = train[neighbors[i],-1]
        if response in votes:
            votes[response] += 1
        else:
            votes[response] = 1
            
    sorted_votes = sorted(votes.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_votes[0][0]


In [37]:
def classify_knn(test, train, k):
    num_rows_test = test.shape[0]
    predictions = []
    for i in range(num_rows_test):
        result = knn(test[i], train, k)
        predictions.append(result)
    return predictions


In [38]:
def normalize(param):
    num_rows = param.shape[0]
    num_cols = param.shape[1]-1
    normalized_params = np.zeros((num_rows, num_cols))
    final_param = param.copy()
    
    for i in range(num_cols):
        normalized_col = (param[:,i] - np.mean(param[:,i])) / np.std(param[:,i])
        normalized_params[:,i] = normalized_col
    final_param[:,:-1] = normalized_params
    return pd.DataFrame(final_param)

def convert_preds_to_df_num(pred):
    df_predictions_knn = pd.DataFrame(pred)
    df_predictions_knn_num = df_predictions_knn.replace(['negative', 'positive'], [0, 1])
    predictions_knn = df_predictions_knn_num.iloc[:,-1]
    return predictions_knn

def knn_neighbor_comparison(arr, nml_test, nml_train):
    performance = {}
    for k in arr:
        X_te, y_te_alpha = split_data_x_y(nml_test)
        y_te = y_te_alpha.replace(['negative', 'positive'], [0, 1])
        
        X_tr = nml_train
        
        prediction_knn = classify_knn(X_te.values, X_tr.values, k)
        predictions_knn_num = convert_preds_to_df_num(prediction_knn)
        
        conf_matrix = get_confusion_matrix(predictions_knn_num.values, y_te.values)
        accuracy = get_accuracy(predictions_knn_num, y_te.values)
        precision = get_precision(conf_matrix.values)
        recall = get_recall(conf_matrix.values)
        f_measure = get_f_measure(precision, recall)
        performance[k] = [accuracy, precision, recall, f_measure]
        
    return performance



In [39]:
all_ks = [1,3,5,25,75,125,225]
normalized_valid = normalize(DS1_valid.values)
normalized_train = normalize(DS1_train.values)

performance = knn_neighbor_comparison(all_ks, normalized_valid, normalized_train)
performance

{1: [0.52375, 0.5375, 0.5231143552311436, 0.530209617755857],
 3: [0.55625, 0.54, 0.5581395348837209, 0.5489199491740787],
 5: [0.54625, 0.52, 0.5488126649076517, 0.5340179717586649],
 25: [0.5375, 0.545, 0.5369458128078818, 0.5409429280397022],
 75: [0.52625, 0.555, 0.524822695035461, 0.5394896719319562],
 125: [0.555, 0.5875, 0.5516431924882629, 0.5690072639225181],
 225: [0.56125, 0.615, 0.5553047404063205, 0.5836298932384341]}

In [42]:
normalized_test = normalize(DS1_test.values)
top_performance = knn_neighbor_comparison([25], normalized_test, normalized_train)
top_performance

{25: [0.5525, 0.5425, 0.5535714285714286, 0.5479797979797979]}

In [43]:
def load_data_ds2_cov():
    numpy_cov_1 = np.genfromtxt("hwk2_datasets/DS2_Cov1.txt", delimiter=',')[:,:-1]
    numpy_cov_2 = np.genfromtxt("hwk2_datasets/DS2_Cov2.txt", delimiter=',')[:,:-1]
    numpy_cov_3 = np.genfromtxt("hwk2_datasets/DS2_Cov3.txt", delimiter=',')[:,:-1]
    
    cov_matrix_1 = pd.DataFrame(numpy_cov_1)
    cov_matrix_2 = pd.DataFrame(numpy_cov_2)
    cov_matrix_3 = pd.DataFrame(numpy_cov_3)
    
    return cov_matrix_1, cov_matrix_2, cov_matrix_3

def load_data_ds2_neg_mean():
    
    negative_mean_1 = np.genfromtxt("hwk2_datasets/DS2_c2_m1.txt", delimiter=',')[:-1]
    negative_mean_2 = np.genfromtxt("hwk2_datasets/DS2_c2_m2.txt", delimiter=',')[:-1]
    negative_mean_3 = np.genfromtxt("hwk2_datasets/DS2_c2_m3.txt", delimiter=',')[:-1]
    
    return negative_mean_1, negative_mean_2, negative_mean_3

def load_data_ds2_pos_mean():

    positive_mean_1 = np.genfromtxt("hwk2_datasets/DS2_c1_m1.txt", delimiter=',')[:-1]
    positive_mean_2 = np.genfromtxt("hwk2_datasets/DS2_c1_m2.txt", delimiter=',')[:-1]
    positive_mean_3 = np.genfromtxt("hwk2_datasets/DS2_c1_m3.txt", delimiter=',')[:-1]
    
    return positive_mean_1, positive_mean_2, positive_mean_3
    

cov_ds2_1, cov_ds2_2, cov_ds2_3 = load_data_ds2_cov()

neg_ds2_mean_1, neg_ds2_mean_2, neg_ds2_mean_3 = load_data_ds2_neg_mean()

pos_ds2_mean_1, pos_ds2_mean_2, pos_ds2_mean_3 = load_data_ds2_pos_mean()


In [44]:
def generate_data_multi_distr(params, coefficients, sample_size):
    if(np.sum(coefficients) != 1):
        raise ValueError("coefficients do not add to 1!")
    
    data = np.zeros((sample_size, 20))
    for i in range(sample_size):
        distr_index = np.random.choice(3, p = coefficients)
        distribution = params[distr_index]
        mean = distribution["mean"]
        cov_mat = distribution["cov"]
        sample_point = generate_data(mean, cov_mat, 1)
        data[i] = sample_point
    return data
        


In [45]:
positive_params = [
    {"mean": pos_ds2_mean_1, "cov": cov_ds2_1},
    {"mean": pos_ds2_mean_2, "cov": cov_ds2_2},
    {"mean": pos_ds2_mean_3, "cov": cov_ds2_3},
]

negative_params = [
    {"mean": neg_ds2_mean_1, "cov": cov_ds2_1},
    {"mean": neg_ds2_mean_2, "cov": cov_ds2_2},
    {"mean": neg_ds2_mean_3, "cov": cov_ds2_3},
]

coefficients = np.array([0.1, 0.42, 0.48])

positive_data_ds2 = generate_data_multi_distr(positive_params, coefficients, 2000)
positive_examples_ds2 = pd.DataFrame(positive_data_ds2)
positive_examples_ds2[20] = 'positive'

negative_data_ds2 = generate_data_multi_distr(negative_params, coefficients, 2000)
negative_examples_ds2 = pd.DataFrame(negative_data_ds2)
negative_examples_ds2[20] = 'negative'


In [46]:
def create_datasets_ds2(neg_ex, pos_ex):
    # train/valid/test split
    seed(42)
    # negative splits
    neg_train, neg_valid, neg_test = train_valid_test_split(neg_ex.values, 0.6, 0.2, 0.2)

    # positive splits
    pos_train, pos_valid, pos_test = train_valid_test_split(pos_ex.values, 0.6, 0.2, 0.2)

    # create datasets
    train = np.concatenate((neg_train, pos_train),axis=0)
    np.random.shuffle(train) 
    train = pd.DataFrame(train)
    
    valid = np.concatenate((neg_valid, pos_valid),axis=0)
    np.random.shuffle(valid) 
    valid = pd.DataFrame(valid)
    
    test = np.concatenate((neg_test, pos_test),axis=0)
    np.random.shuffle(test) 
    test = pd.DataFrame(test)
    
    return train, valid, test

In [47]:
DS2_train, DS2_valid, DS2_test = create_datasets_ds2(negative_examples_ds2, positive_examples_ds2)

DS2_train.to_csv("DS2_train.csv")
DS2_valid.to_csv("DS2_valid.csv")
DS2_test.to_csv("DS2_test.csv")

In [48]:
X2_train, y2_train_alpha = split_data_x_y(DS2_train)
y2_train = y2_train_alpha.replace(['negative', 'positive'], [0, 1])

X2_valid, y2_valid_alpha = split_data_x_y(DS2_valid)
y2_valid = y2_valid_alpha.replace(['negative', 'positive'], [0, 1])

X2_test, y2_test_alpha = split_data_x_y(DS2_test)
y2_test = y2_test_alpha.replace(['negative', 'positive'], [0, 1])


In [49]:
pi_prior_2, m0_2, m1_2, sigma_2 = gda_parameters_maximum_likelihood(X2_train.values, y2_train.values)

In [50]:
predictions_2 = classify(X2_test.values, m0_2, m1_2, sigma_2, pi_prior_2)

Class 1:
[[0.04538512]]

[[ 0.00398763]
 [-0.05683509]
 [-0.07155181]
 [-0.01016242]
 [ 0.04295847]
 [-0.0485771 ]
 [-0.01398687]
 [ 0.003473  ]
 [ 0.03296811]
 [ 0.0215139 ]
 [-0.06979787]
 [ 0.04639429]
 [ 0.05374306]
 [ 0.03564288]
 [ 0.0232841 ]
 [-0.02778823]
 [-0.03966765]
 [-0.03637554]
 [ 0.02401795]
 [ 0.04673235]]

Class 0:
[[-0.04538512]]

[[-0.00398763]
 [ 0.05683509]
 [ 0.07155181]
 [ 0.01016242]
 [-0.04295847]
 [ 0.0485771 ]
 [ 0.01398687]
 [-0.003473  ]
 [-0.03296811]
 [-0.0215139 ]
 [ 0.06979787]
 [-0.04639429]
 [-0.05374306]
 [-0.03564288]
 [-0.0232841 ]
 [ 0.02778823]
 [ 0.03966765]
 [ 0.03637554]
 [-0.02401795]
 [-0.04673235]]



In [51]:
confusion_matrix_2 = get_confusion_matrix(predictions_2, y2_test.values)

accuracy_2 = get_accuracy(predictions_2, y2_test.values)
precision_2 = get_precision(confusion_matrix_2.values)
recall_2 = get_recall(confusion_matrix_2.values)
f_measure_2 = get_f_measure(precision_2, recall_2)

accuracy_2, precision_2, recall_2, f_measure_2

(0.52625, 0.495, 0.528, 0.5109677419354839)

In [52]:
all_ks_2 = [1,3,5,25,75,125,175,225]
normalized_valid_2 = normalize(DS2_valid.values)
normalized_train_2 = normalize(DS2_train.values)

performance_2 = knn_neighbor_comparison(all_ks_2, normalized_valid_2, normalized_train_2)
performance_2

{1: [0.48875, 0.45, 0.4878048780487805, 0.46814044213263983],
 3: [0.53125, 0.5275, 0.5314861460957179, 0.5294855708908407],
 5: [0.51125, 0.5, 0.5115089514066496, 0.5056890012642226],
 25: [0.5075, 0.4725, 0.5080645161290323, 0.4896373056994819],
 75: [0.49, 0.4075, 0.4880239520958084, 0.444141689373297],
 125: [0.50875, 0.41, 0.5109034267912772, 0.45492371705963935],
 175: [0.51, 0.3725, 0.5137931034482759, 0.4318840579710145],
 225: [0.5025, 0.365, 0.503448275862069, 0.4231884057971015]}

In [53]:
normalized_test_2 = normalize(DS2_test.values)
top_performance_2 = knn_neighbor_comparison([25], normalized_test_2, normalized_train_2)
top_performance_2

{25: [0.5475, 0.515, 0.5508021390374331, 0.5322997416020671]}